# Practical 6
#### [ ID: 17CE016 ]
---
#### Aim: Predicting Sports Winners with Decision Trees

#### Theory:
##### Introduction to Decision Trees :
- A decision tree is a decision support tool that uses a tree-like graph or model of decisions and their possible consequences, including chance event outcomes, resource costs, and utility. It is one way to display an algorithm that only contains conditional control statements.
- A decision tree is a flowchart-like structure in which each internal node represents a “test” on an attribute (e.g. whether a coin flip comes up heads or tails), each branch represents the outcome of the test, and each leaf node represents a class label (decision taken after computing all attributes). The paths from root to leaf represent classification rules.
- Tree based learning algorithms are considered to be one of the best and mostly used supervised learning methods. Tree based methods empower predictive models with high accuracy, stability and ease of interpretation. Unlike linear models, they map non-linear relationships quite well. They are adaptable at solving any kind of problem at hand (classification or regression). Decision Tree algorithms are referred to as CART (Classification and Regression Trees).
- Methods like decision trees, random forest, gradient boosting are being popularly used in all kinds of data science problems.


---

In [1]:
import pandas as pd

data_filename = "basketball.csv"
dataset = pd.read_csv(data_filename)

dataset.head() #print top 5 rows

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes
0,Tue Oct 27 2015,8:00p,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,19187,NaN
1,Tue Oct 27 2015,8:00p,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,21957,NaN
2,Tue Oct 27 2015,10:30p,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,19596,NaN
3,Wed Oct 28 2015,7:30p,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,18624,NaN
4,Wed Oct 28 2015,7:30p,Chicago Bulls,115,Brooklyn Nets,100,Box Score,NaN,17732,NaN


In [2]:
#format the date column
dataset = pd.read_csv(data_filename, parse_dates=["Date"])
#label the columns of the dataset
dataset.columns = ["Date", "Start (ET)", "Visitor Team", "VisitorPts", "Home Team", "HomePts", "OT?", "Score Type", "Attend.", "Notes"]

dataset.head() #print top 5 rows

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Score Type,Attend.,Notes
0,2015-10-27,8:00p,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,19187,NaN
1,2015-10-27,8:00p,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,21957,NaN
2,2015-10-27,10:30p,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,19596,NaN
3,2015-10-28,7:30p,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,18624,NaN
4,2015-10-28,7:30p,Chicago Bulls,115,Brooklyn Nets,100,Box Score,NaN,17732,NaN


In [3]:
print(dataset.dtypes) #print datatype of each column of dataset

dataset["HomeWin"] = dataset["VisitorPts"] < dataset["HomePts"]  #add new feature/column depending on who has more points

dataset.head()

Date            datetime64[ns]
Start (ET)              object
Visitor Team            object
VisitorPts               int64
Home Team               object
HomePts                  int64
OT?                     object
Score Type              object
Attend.                  int64
Notes                   object
dtype: object


,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Score Type,Attend.,Notes,HomeWin
0,2015-10-27,8:00p,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,19187,NaN,False
1,2015-10-27,8:00p,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,21957,NaN,True
2,2015-10-27,10:30p,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,19596,NaN,True
3,2015-10-28,7:30p,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,18624,NaN,True
4,2015-10-28,7:30p,Chicago Bulls,115,Brooklyn Nets,100,Box Score,NaN,17732,NaN,False


In [4]:
y_true = dataset["HomeWin"].values #create list of class for homewinner
print(type(y_true))

dataset["HomeWin"].mean()  #mean of homewinner chances

from collections import defaultdict
won_last = defaultdict(int)

#create two new datsets
dataset["HomeLastWin"] = 0  
dataset["VisitorLastWin"] = 0

<class 'numpy.ndarray'>


In [5]:
#for each row , update the winner of previous game 

for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeLastWin"] = won_last[home_team]
    dataset.at[index, "HomeLastWin"] = won_last[home_team]
    dataset.at[index, "VisitorLastWin"] =  won_last[visitor_team]
#also, create two more class list for last time winners    
    won_last[home_team] = int(row["HomeWin"])
    won_last[visitor_team] = 1 - int(row["HomeWin"])
    
dataset.tail(6)

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Score Type,Attend.,Notes,HomeWin,HomeLastWin,VisitorLastWin
1310,2016-06-05,8:00p,Cleveland Cavaliers,77,Golden State Warriors,110,Box Score,NaN,19596,NaN,True,1,0
1311,2016-06-08,9:00p,Golden State Warriors,90,Cleveland Cavaliers,120,Box Score,NaN,20562,NaN,True,0,1
1312,2016-06-10,9:00p,Golden State Warriors,108,Cleveland Cavaliers,97,Box Score,NaN,20562,NaN,False,1,0
1313,2016-06-13,9:00p,Cleveland Cavaliers,112,Golden State Warriors,97,Box Score,NaN,19596,NaN,False,1,0
1314,2016-06-16,9:00p,Golden State Warriors,101,Cleveland Cavaliers,115,Box Score,NaN,20562,NaN,True,1,0
1315,2016-06-19,8:00p,Cleveland Cavaliers,93,Golden State Warriors,89,Box Score,NaN,19596,NaN,False,0,1


In [6]:
dataset.iloc[1000:1005]

#create numpy array 
X_previouswins = dataset[["HomeLastWin", "VisitorLastWin"]].values
print(type(X_previouswins))

<class 'numpy.ndarray'>


---

### Decision Tree

In [7]:
#################### decision tree application starts ##################

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=14)

from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(clf, X_previouswins, y_true,scoring='accuracy',cv=3)
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 59.4%


In [8]:
#testing with different feature
import os
standings_filename = os.path.join("standings.csv")

standings = pd.read_csv(standings_filename, skiprows=1)

standings.head()

,Rk,Team,Overall,Home,Road,E,W,A,C,SE,...,Post,=3,=10,Oct,Nov,Dec,Jan,Feb,Mar,Apr
0,1,Golden State Warriors,67-15,39-2,28-13,25-5,42-10,9-1,7-3,9-1,...,25-6,5-3,45-9,1-0,13-2,11-3,12-3,8-3,16-2,6-2
1,2,Atlanta Hawks,60-22,35-6,25-16,38-14,22-8,12-6,14-4,12-4,...,17-11,6-4,30-10,0-1,9-5,14-2,17-0,7-4,9-7,4-3
2,3,Houston Rockets,56-26,30-11,26-15,23-7,33-19,9-1,8-2,6-4,...,20-9,8-4,31-14,2-0,11-4,9-5,11-6,7-3,10-6,6-2
3,4,Los Angeles Clippers,56-26,30-11,26-15,19-11,37-15,7-3,6-4,6-4,...,21-7,3-5,33-9,2-0,9-5,11-6,11-4,5-6,11-5,7-0
4,5,Memphis Grizzlies,55-27,31-10,24-17,20-10,35-17,8-2,5-5,7-3,...,16-13,9-3,26-13,2-0,13-2,8-6,12-4,7-4,9-8,4-3


In [9]:
dataset["HomeTeamRanksHigher"] = 0
for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    home_rank = standings[standings["Team"] == home_team]["Rk"].values[0]
    visitor_rank = standings[standings["Team"] == visitor_team]["Rk"].values[0]
    dataset.at[index, "HomeTeamRanksHigher"] =  int(home_rank < visitor_rank)
    
X_homehigher = dataset[[ "HomeTeamRanksHigher", "HomeLastWin", "VisitorLastWin",]].values

In [10]:
clf = DecisionTreeClassifier(random_state=14, criterion="entropy")

scores = cross_val_score(clf, X_homehigher, y_true, scoring='accuracy',cv=3)

print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 60.9%


In [11]:
last_match_winner = defaultdict(int)
dataset["HomeTeamWonLast"] = 0

for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    
    teams = tuple(sorted([home_team, visitor_team]))  # Sort for a consistent ordering

    # Set in the row, who won the last encounter
    home_team_won_last = 1 if last_match_winner[teams] == row["Home Team"] else 0
    dataset.at[index, "HomeTeamWonLast"] =  home_team_won_last
    # Who won this one?
    winner = row["Home Team"] if row["HomeWin"] else row["Visitor Team"]
    last_match_winner[teams] = winner

In [12]:
dataset.loc[400:420]

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Score Type,Attend.,Notes,HomeWin,HomeLastWin,VisitorLastWin,HomeTeamRanksHigher,HomeTeamWonLast
400,2015-12-19,5:00p,Los Angeles Lakers,78,Oklahoma City Thunder,118,Box Score,NaN,18203,NaN,True,0,0,1,0
401,2015-12-19,7:00p,Charlotte Hornets,101,Washington Wizards,109,Box Score,NaN,16987,NaN,True,0,1,1,0
402,2015-12-20,1:00p,Minnesota Timberwolves,100,Brooklyn Nets,85,Box Score,NaN,14552,NaN,False,0,1,1,0
403,2015-12-20,3:30p,Philadelphia 76ers,86,Cleveland Cavaliers,108,Box Score,NaN,20562,NaN,True,1,0,1,1
404,2015-12-20,8:00p,New Orleans Pelicans,130,Denver Nuggets,125,Box Score,NaN,13857,NaN,False,0,0,0,1
405,2015-12-20,1:00p,Portland Trail Blazers,109,Miami Heat,116,Box Score,NaN,19600,NaN,True,0,0,0,0
406,2015-12-20,6:00p,Atlanta Hawks,103,Orlando Magic,100,Box Score,NaN,16982,NaN,False,1,1,0,0
407,2015-12-20,5:00p,Milwaukee Bucks,101,Phoenix Suns,95,Box Score,NaN,16859,NaN,False,1,0,0,0
408,2015-12-20,6:00p,Sacramento Kings,104,Toronto Raptors,94,Box Score,NaN,19800,NaN,False,1,0,1,0
409,2015-12-21,8:00p,Portland Trail Blazers,97,Atlanta Hawks,106,Box Score,NaN,18373,NaN,True,1,0,1,0


In [13]:
X_lastwinner = dataset[[ "HomeTeamWonLast", "HomeTeamRanksHigher", "HomeLastWin", "VisitorLastWin",]].values
clf = DecisionTreeClassifier(random_state=14, criterion="entropy")

scores = cross_val_score(clf, X_lastwinner, y_true, scoring='accuracy',cv=3)

print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 62.2%


In [14]:
from sklearn.preprocessing import LabelEncoder
encoding = LabelEncoder()
encoding.fit(dataset["Home Team"].values)
home_teams = encoding.transform(dataset["Home Team"].values)

visitor_teams = encoding.transform(dataset["Visitor Team"].values)
X_teams = np.vstack([home_teams, visitor_teams]).T


from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories='auto')
X_teams = onehot.fit_transform(X_teams).todense()

clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy',cv=3)
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 62.8%


### Random Forest

In [15]:
# random forest 
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=14 , n_estimators = 10)
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy', cv=3)
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 65.3%


In [16]:
# with all the features
X_all = np.hstack([X_lastwinner, X_teams])
clf = RandomForestClassifier(random_state=14, n_estimators = 10)
scores = cross_val_score(clf, X_all, y_true, scoring='accuracy' , cv=3)
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 63.3%


In [17]:
#with estimators 250
X_all = np.hstack([X_lastwinner, X_teams])
clf = RandomForestClassifier(random_state=14, n_estimators=250)
scores = cross_val_score(clf, X_all, y_true, scoring='accuracy' , cv=3)
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 64.5%


In [18]:
from sklearn.model_selection import GridSearchCV
parameter_space = {
    "max_features": [2, 10, 'auto'],
    "n_estimators": [100, 200],
    "criterion": ["gini", "entropy"],
    "min_samples_leaf": [2, 4, 6],
}
clf = RandomForestClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space , cv=3)
grid.fit(X_all, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))

Accuracy: 67.4%


In [19]:
print(grid.best_estimator_)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=14, verbose=0,
                       warm_start=False)


---
## Conclusion:
In this practical, I have learnt how to predict the sport winner using methods like decision tree and random forests. I have compared both the methods using different features and 
found their accuracy at different stages.

---